In [13]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# --- Krok 1: Definicje Środowiska i Jednostek ---

# Ustawienia "pola bitwy" (Pana "rozdzielczość pola testów")
# Zakładamy, że jednostki są w metrach.
WORLD_SETTINGS = {
    'width': 1000,  # metry
    'height': 1000  # metry
}

# Elastyczna definicja TYPÓW jednostek (łatwe dodawanie nowych)
# Tutaj definiujemy "szablony"
SYSTEM_TYPES = {
    "C-RAM": {
        "range": 250,         # Zasięg skuteczny w metrach
        "service_time": 5.0,  # Czas "obsługi" celu w sekundach
        "color": 'blue'       # Kolor na wizualizacji
    },
    "LASER": {
        "range": 400,
        "service_time": 3.0,
        "color": 'red'
    }
    # Można tu dodać np. "JAMMER", "MANPADS" itp.
}

# Definicja konkretnych JEDNOSTEK rozmieszczonych na polu bitwy
# Każda jednostka ma swoje ID, bazuje na TYPIE i ma konkretną pozycję (x, y)
BATTLEFIELD_UNITS = [
    {
        "id": "cram_1",
        "type": "C-RAM",
        "position": (400, 500)
    },
    {
        "id": "cram_2",
        "type": "C-RAM",
        "position": (400, 700)
    },
    {
        "id": "laser_1",
        "type": "LASER",
        "position": (600, 600)
    }
]

# --- Krok 2: Funkcja Wizualizująca ---

def visualize_static_setup(world, types, units):
    """
    Tworzy statyczny wykres 2D pokazujący rozmieszczenie jednostek
    i ich zasięg na polu bitwy.
    """
    
    # Inicjalizacja wykresu
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_aspect('equal') # Kluczowe dla zachowania proporcji i okręgów

    # Ustawienie granic "świata"
    ax.set_xlim(0, world['width'])
    ax.set_ylim(0, world['height'])
    
    # Ustawienia estetyczne
    ax.set_title("Rozmieszczenie Systemów Obronnych")
    ax.set_xlabel("Pozycja X (metry)")
    ax.set_ylabel("Pozycja Y (metry)")
    ax.grid(True, linestyle=':', alpha=0.6)

    legend_handles = [] # Do zebrania elementów legendy

    # Iterujemy po jednostkach na mapie
    for unit in units:
        # Pobieramy pozycję
        x, y = unit['position']
        
        # Pobieramy właściwości z definicji TYPU
        unit_type = unit['type']
        type_properties = types[unit_type]
        
        unit_range = type_properties['range']
        unit_color = type_properties['color']
        
        # 1. Rysujemy pozycję jednostki (jako 'X')
        ax.plot(x, y, 'x', color=unit_color, markersize=10, markeredgewidth=2,
                label=f"Jednostka: {unit['id']} ({unit_type})")
        
        # 2. Rysujemy zasięg (okrąg)
        # Używamy 'patches.Circle'
        zone = patches.Circle((x, y), unit_range, 
                              facecolor=unit_color, 
                              alpha=0.1,  # Przezroczystość
                              edgecolor=unit_color, 
                              linestyle='--')
        ax.add_patch(zone)

    # Tworzymy legendę (może być długa, jeśli jest wiele jednostek)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))
    plt.tight_layout() # Dopasowuje wykres
    
    # Wyświetlamy wykres
    plt.show()

# --- Uruchomienie wizualizacji ---
if __name__ == "__main__":
    print("Generowanie statycznej mapy pola bitwy...")
    visualize_static_setup(WORLD_SETTINGS, SYSTEM_TYPES, BATTLEFIELD_UNITS)


Generowanie statycznej mapy pola bitwy...


In [14]:



import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.animation as animation
import time

# --- Krok 1: Definicje Środowiska i Systemów Obronnych (Bez zmian) ---

WORLD_SETTINGS = {
    'width': 1000,  # metry
    'height': 1000  # metry
}

SYSTEM_TYPES = {
    "C-RAM": {
        "range": 250,
        "service_time": 5.0,
        "color": 'blue'
    },
    "LASER": {
        "range": 400,
        "service_time": 3.0,
        "color": 'red'
    }
}

BATTLEFIELD_UNITS = [
    {"id": "cram_1", "type": "C-RAM", "position": (400, 500)},
    {"id": "cram_2", "type": "C-RAM", "position": (400, 700)},
    {"id": "laser_1", "type": "LASER", "position": (600, 600)}
]

# --- Krok 2: Nowe Definicje (Drony i Scenariusz Ataku) ---

# Elastyczna definicja TYPÓW dronów
DRONE_TYPES = {
    "Scout": {
        "speed": 60,  # m/s (szybszy)
        "color": 'blue'
    },
    "Bomber": {
        "speed": 35,  # m/s (wolniejszy)
        "color": 'gray'
    }
}

# Definicja punktów trasy (zgodnie z Pana prośbą)
SPAWN_START_POS = (500, 1000) # (x, y)
TARGET_POS = (500, 0)      # (x, y)

# Definicja fali dronów (łatwa do modyfikacji)
# Lista "paczek" do wysłania.
SPAWN_WAVE = [
    # Paczka 1
    {
        "type": "Scout",  # Jaki typ drona
        "count": 10,      # Ile sztuk
        "interval": 3.0   # Co ile sekund wysyłać (stały odstęp)
    },
    # Paczka 2 (uruchomi się po Paczce 1)
    {
        "type": "Bomber",
        "count": 5,
        "interval": 5.0
    }
]



In [15]:
# --- Krok 3: Klasy Obiektów Symulacji ---

class Drone:
    """Reprezentuje pojedynczego drona (klienta)."""
    def __init__(self, x, y, drone_type_name, target_x, target_y):
        self.x = x
        self.y = y
        self.type_name = drone_type_name
        self.props = DRONE_TYPES[drone_type_name]
        self.speed = self.props['speed']
        self.color = self.props['color']
        self.target_x = target_x
        self.target_y = target_y
        
        self.status = 'active'  # 'active', 'leaked', 'destroyed'
        
        # Obliczanie wektora prędkości (vx, vy)
        dx = self.target_x - self.x
        dy = self.target_y - self.y
        dist = np.hypot(dx, dy)
        
        if dist > 0:
            # Normalizujemy wektor kierunku i mnożymy przez prędkość
            self.vx = (dx / dist) * self.speed
            self.vy = (dy / dist) * self.speed
        else:
            self.vx = 0
            self.vy = 0

    def move(self, dt):
        """Przesuwa drona na podstawie jego prędkości i kroku czasowego dt."""
        if self.status == 'active':
            self.x += self.vx * dt
            self.y += self.vy * dt
            self.check_status()

    def check_status(self):
        """Sprawdza, czy dron dotarł do celu."""
        # Sprawdzamy, czy dron przeleciał przez linię celu
        # Ponieważ leci w dół (vy < 0), sprawdzamy warunek 'mniejsze lub równe'
        if self.vy < 0 and self.y <= self.target_y:
            self.status = 'leaked'
        # TODO: Dodać logikę dla innych kierunków lotu, jeśli będą potrzebne


class Simulation:
    """Zarządza całą symulacją, obiektami i pętlą czasu."""
    def __init__(self, world, systems, units, wave, start_pos, target_pos):
        self.world = world
        self.system_types = systems
        self.battlefield_units = units
        
        self.start_pos = start_pos
        self.target_pos = target_pos
        
        self.sim_time = 0.0
        # Krok czasowy. dt=0.1 oznacza 10 klatek symulacji na sekundę.
        self.dt = 0.1 

        self.active_drones = []
        self.leaked_drones = []
        self.destroyed_drones = [] # Na razie pusta, ale gotowa na Krok 3
        
        # --- Logika Spawnera ---
        self.spawn_plan = list(wave) # Kopia planu fali
        self.next_spawn_time = 0.0
        self.spawn_interval = 0.0
        self.drones_to_spawn_in_batch = 0
        self.current_batch_type = None
        self._setup_next_batch() # Ustawiamy pierwszą paczkę

    def _setup_next_batch(self):
        """Pobiera następną paczkę z planu fali."""
        if not self.spawn_plan:
            # Koniec planu, nic więcej nie wysyłamy
            self.drones_to_spawn_in_batch = 0
            print(f"[{self.sim_time:.1f}s] Wszystkie fale dronów zostały wysłane.")
            return

        batch = self.spawn_plan.pop(0) # Pobieramy pierwszą paczkę z listy
        self.current_batch_type = batch['type']
        self.drones_to_spawn_in_batch = batch['count']
        self.spawn_interval = batch['interval']
        # Ustawiamy czas spawnu pierwszego drona z tej paczki
        self.next_spawn_time = self.sim_time + self.spawn_interval 
        print(f"[{self.sim_time:.1f}s] Rozpoczynanie fali: {self.current_batch_type} ({self.drones_to_spawn_in_batch} szt. co {self.spawn_interval}s)")

    def _spawn_drone(self):
        """Tworzy nowego drona na podstawie bieżącej paczki."""
        x, y = self.start_pos
        new_drone = Drone(x, y, self.current_batch_type, self.target_pos[0], self.target_pos[1])
        self.active_drones.append(new_drone)
        
        self.drones_to_spawn_in_batch -= 1
        self.next_spawn_time += self.spawn_interval # Następny za X sekund
        
        # Sprawdź, czy ta paczka się skończyła
        if self.drones_to_spawn_in_batch <= 0:
            self._setup_next_batch()

    def step(self):
        """Wykonuje jeden krok symulacji (o długości 'dt')."""
        
        # 1. Spawning
        # Sprawdzamy, czy mamy coś do wysłania i czy nadszedł czas
        if self.drones_to_spawn_in_batch > 0 and self.sim_time >= self.next_spawn_time:
            self._spawn_drone()

        # 2. Przesuwanie dronów
        # Iterujemy po kopii listy [:] aby móc bezpiecznie usuwać z oryginału
        for drone in self.active_drones[:]:
            drone.move(self.dt)
            
            # 3. Sprawdzanie statusu dronów
            if drone.status == 'leaked':
                print(f"[{self.sim_time:.1f}s] Dron {drone.type_name} przeciekł (osiągnął cel).")
                self.active_drones.remove(drone)
                self.leaked_drones.append(drone)
        
        # 4. TODO: Logika systemów obronnych (Krok 3)
        # Na razie nic nie robią
        
        # 5. Inkrementacja czasu
        self.sim_time += self.dt

    def is_finished(self):
        """Sprawdza, czy symulacja dobiegła końca."""
        # Koniec, gdy nie ma już aktywnych dronów i nic więcej nie przyleci
        no_active = not self.active_drones
        no_more_spawns = self.drones_to_spawn_in_batch <= 0 and not self.spawn_plan
        return no_active and no_more_spawns



In [16]:
%matplotlib qt
# --- Krok 4: Wizualizacja (Animacja) ---

# Globalne obiekty fig, ax i ani
fig, ax = plt.subplots(figsize=(10, 10))
# Inicjalizacja głównej klasy symulacji
sim = Simulation(WORLD_SETTINGS, SYSTEM_TYPES, BATTLEFIELD_UNITS, 
                 SPAWN_WAVE, SPAWN_START_POS, TARGET_POS)
ani = None # Zmienna przechowująca obiekt animacji

def draw_static_environment(ax):
    """Pomocnicza funkcja do rysowania tła (mapy i jednostek)."""
    
    # Ustawienia osi
    ax.set_aspect('equal')
    ax.set_xlim(0, sim.world['width'])
    ax.set_ylim(0, sim.world['height'])
    ax.grid(True, linestyle=':', alpha=0.6)
    
    # Linia celu (tam gdzie drony mają dolecieć)
    ax.axhline(sim.target_pos[1], color='black', linestyle='--', linewidth=2,
                label=f"Linia Celu (y={sim.target_pos[1]})")

    # Rysowanie jednostek obronnych i ich zasięgu
    for unit in sim.battlefield_units:
        x, y = unit['position']
        props = sim.system_types[unit['type']]
        unit_range = props['range']
        unit_color = props['color']
        
        # Rysujemy pozycję (X)
        ax.plot(x, y, 'x', color=unit_color, markersize=10, markeredgewidth=2,
                label=f"Jednostka: {unit['id']} ({unit['type']})")
        
        # Rysujemy zasięg (okrąg)
        zone = patches.Circle((x, y), unit_range, 
                              facecolor=unit_color, alpha=0.1, 
                              edgecolor=unit_color, linestyle='--')
        ax.add_patch(zone)
    
    # Legenda dla jednostek
    ax.legend(loc='upper right')

def update_plot(frame):
    """Główna funkcja animacji, wywoływana cyklicznie."""
    global ani
    
    # 1. Wykonaj krok symulacji
    if not sim.is_finished():
        sim.step()
    
    # 2. Wyczyść poprzednią klatkę
    ax.clear()
    
    # 3. Narysuj statyczne tło (jednostki, strefy, cel)
    draw_static_environment(ax)
    
    # 4. Narysuj aktywne drony
    for drone in sim.active_drones:
        # Rysujemy drona jako kropkę 'o'
        ax.plot(drone.x, drone.y, 'o', color=drone.color, markersize=6)
            
    # 5. Narysuj drony, które przeciekły (jako 'duchy' na linii celu)
    for drone in sim.leaked_drones:
        # Rysujemy je jako trójkąty 'v' na linii celu
        ax.plot(drone.x, drone.target_y, 'v', color='gray', markersize=6, alpha=0.6)

    # 6. Ustaw tytuł i etykiety
    ax.set_title(f"Symulacja Obrony [Krok 2: Ruch Dronów]\n"
                 f"Czas: {sim.sim_time:.1f}s | "
                 f"Przeciekło: {len(sim.leaked_drones)}")
    ax.set_xlabel("Pozycja X (metry)")
    ax.set_ylabel("Pozycja Y (metry)")

    # 7. Zakończ animację, jeśli symulacja się skończyła
    if sim.is_finished():
        print("--- Symulacja [Krok 2] zakończona ---")
        print(f"Całkowity czas: {sim.sim_time:.1f}s")
        print(f"Całkowity przeciek: {len(sim.leaked_drones)}")
        if ani:
            ani.event_source.stop() # Zatrzymuje pętlę animacji

# --- Uruchomienie ---
def main():
    global ani
    print("Uruchamianie animacji [Krok 2: Symulacja lotu dronów]...")
    print("Systemy obronne są na razie BEZCZYNNE.")
    
    # interval = 1000 * dt (np. 1000 * 0.1s = 100ms)
    # To zapewnia, że animacja działa w czasie rzeczywistym
    # (1 sekunda symulacji = 1 sekunda w realnym świecie)
    ani = animation.FuncAnimation(fig, update_plot, 
                                  interval=(sim.dt * 1000), 
                                  blit=False)
    
    plt.show() # Otwiera okno animacji

if __name__ == "__main__":
    main()

[0.0s] Rozpoczynanie fali: Scout (10 szt. co 3.0s)
Uruchamianie animacji [Krok 2: Symulacja lotu dronów]...
Systemy obronne są na razie BEZCZYNNE.


C:\Users\mrmro\AppData\Local\Temp\ipykernel_2828\226851209.py:92: UserWarning: frames=None which we can infer the length of, did not pass an explicit *save_count* and passed cache_frame_data=True.  To avoid a possibly unbounded cache, frame data caching has been disabled. To suppress this warning either pass `cache_frame_data=False` or `save_count=MAX_FRAMES`.
  ani = animation.FuncAnimation(fig, update_plot,
